In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 224, 224
train_data_dir = 'grayscale_dataset/train'
test_data_dir = 'grayscale_dataset/test'
epochs = 10
batch_size = 64

model = Sequential()
model.add(Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Conv2D(256, (11, 11), strides=(1, 1), padding='valid', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')

# Train the model
try:
    model.fit(train_generator,
              steps_per_epoch=train_generator.samples // batch_size,
              epochs=epochs,
              validation_data=test_generator,
              validation_steps=test_generator.samples // batch_size)
except Exception as e:
    print("Error occurred during training:", e)

loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
